In [1]:
class Graph:
    def __init__(self):
        self.adj_dict = {}
        
    def add_edge(self, v, u, pattern) :
        if v not in self.adj_dict:
            self.adj_dict[v] = {u:pattern}
        else:
            self.adj_dict[v][u] = pattern
            
        if u not in self.adj_dict:
            self.adj_dict[u] = {v:pattern}
        else:
            self.adj_dict[u][v] = pattern
        
    def adj(self, v):
        return self.adj_dict[v]
    
    def vertex(self, v):
        self.adj_list = self.adj_dict[v]
        self.deg = sum(list(map(int, self.adj_list.values())))
        self.vertex_num = 1
        
        return self.vertex_num, self.adj_list, self.deg

In [2]:
import sys
import time

edges_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.edges' #sys.argv[1]
label_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.node_labels' #sys.argv[2]
# k = 7 #sys.argv[3]
label_dict = {}
graph_sort = []
label_kind = []
node_dict = {}
pt_fq = {} # 빈발패턴
label_f = open(label_path, 'r')
while True:
    line = label_f.readline().replace('\n', '').strip()
    if not line:
        label_kind.append('0')
        break
    
    node = line.split(',')[0]
    label = line.split(',')[1]
    label_dict[node] = label
    label_kind.append(label)
label_f.close

label_kind = list(set(label_kind))

edge_f = open(edges_path, 'r')
gs = Graph()
while True:
    line = edge_f.readline().replace('\n', '').strip()
    if not line:
        break
    
    v = line.split(',')[0]
    u = line.split(',')[1]
    if v not in label_dict:
        label_dict[v] = '0'
        
    if u not in label_dict:
        label_dict[u] = '0'
    graph_sort.append([v,u])
    
    if v not in node_dict:
        node_dict[v] = v
    if u not in node_dict:
        node_dict[u] = u
        
    pt = label_dict[v] + '/' + label_dict[u]
    re_pt = ''.join(reversed(pt))
    
    if pt not in pt_fq and re_pt not in pt_fq:
        pt_fq[pt] = 1
        gs.add_edge(v,u,pt)
        
    elif re_pt in pt_fq:
        pt_fq[re_pt] = pt_fq[re_pt] + 1
        gs.add_edge(v,u,re_pt)
        
    elif pt in pt_fq:
        pt_fq[pt] = pt_fq[pt] + 1
        gs.add_edge(v,u,pt)

    
edge_f.close

<function TextIOWrapper.close()>

In [7]:
gs.adj('1')

{'2': '1/0',
 '3': '1/1',
 '4': '1/0',
 '5': '1/1',
 '6': '1/0',
 '7': '1/0',
 '8': '1/2',
 '9': '1/0',
 '10': '1/0',
 '11': '1/0',
 '12': '1/0',
 '13': '1/4',
 '14': '1/0',
 '15': '1/28',
 '16': '1/0',
 '17': '1/0',
 '18': '1/0',
 '19': '1/0',
 '20': '1/0',
 '21': '1/0',
 '22': '1/0',
 '376': '1/5',
 '1219': '1/16',
 '268635': '1/0',
 '317880': '1/0',
 '645927': '1/0',
 '645928': '1/0',
 '645929': '1/0',
 '645930': '1/0'}

In [71]:
a = list(pt_fq.values())
sum = 0
for n in a:
    sum = sum + n

mean = sum / len(a)
print(mean)

1648.5352811466373


In [72]:
print(len(pt_fq))
print(len(pt_fq)*99//100)
print(len(pt_fq)%10)
print(20/len(pt_fq))
n = 99
value = list(sorted(pt_fq.items(), key = lambda item: item[1], reverse = True))
# print(value)
new_pt_fq = {}
new_pt_fq.update(pt_fq)

1814
1795
4
0.011025358324145534


In [74]:
p_id = 0
Freq = 0
TF = 0
FR = 0

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = len(dict)*n//100 - 1
    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

def pattern_gen(ptternFrequency, pttern_th):
    add_pt = {}
    
    for pt1 in ptternFrequency:
        if ptternFrequency[pt1] >= pttern_th:
            pt_tgt = pt1.split('/')
            for pt2 in ptternFrequency:
                if pt1 != pt2 and ptternFrequency[pt2] >= pttern_th:
                    pt_sce = pt2.split('/')
                    between_len = len(pt_tgt) - 2
                    if between_len == 0:
                        if pt_tgt[0] == pt_sce[0]:
                            pt = pt_sce[1] + '/' + pt1
                        elif pt_tgt[0] == pt_sce[1]:
                            pt = pt_sce[0] + '/' + pt1
                        elif pt_tgt[-1] == pt_sce[0]:
                            pt = pt1 + '/' + pt_sce[1]

                        elif pt_tgt[-1] == pt_sce[-1]:
                            pt = pt1 + '/' + pt_sce[0]
                        else:
                            continue
                    else:
                        if pt_tgt[:between_len] == pt_sce[between_len:-1]: #앞, 뒤
                            pt = pt_sce[0] + '/' + pt1
                        elif pt_tgt[between_len:-1] == pt_sce[:between_len]: #뒤, 앞
                            pt = pt1 + '/' + pt_sce[-1]
                        else:
                            continue
                        
                    re_pt = ''.join(reversed(pt))
        
                    if pt not in add_pt and re_pt not in add_pt:
                        add_pt[pt] = min(ptternFrequency[pt1], ptternFrequency[pt2])
    
    return add_pt

pers = 10
def updatePattern(updateDict, pers):
    update_dict = {}
    update_dict = (pattern_gen(updateDict, Top_nPercent(updateDict, pers)))
    if len(update_dict) != 0:
        new_pt_fq.update(update_dict)
        updatePattern(update_dict, pers)
    
updatePattern(pt_fq, pers)
print(len(pt_fq))
print(len(new_pt_fq))
    
# print('a')
# a = pattern_gen(pt_fq, Top_nPercent(pt_fq, pers))
# print(len(a), Top_nPercent(pt_fq, pers))
# print('b')
# b = pattern_gen(a, Top_nPercent(a, pers))
# print(len(b), Top_nPercent(a, pers))
# print('c')
# c = pattern_gen(b, Top_nPercent(b, pers))
# print(len(c), Top_nPercent(b, pers))

# print(sorted(b.items(), key = lambda item: item[1], reverse = True))
            

0
1814
6947


In [ ]:
print(len(pt_fq))
print(len(add_pt))
pt_fq.update(add_pt)
print(len(pt_fq))

print(sorted(pt_fq.items(), key = lambda item: item[1], reverse = True))

In [ ]:
p_id = 0
Freq = 0
TF = 0
FR = 0
pattern_list = []

pattern_list.append([p_id, pt, Freq, TF, FR])


for p_one in pattern_list:
    if p_one[1] == pt or p_one[1] == re_pt:
        p_id = pattern_list.index(p_one)
        Freq = p_one[2] + 1
    pattern_list.append([p_id, pt, Freq, TF, FR])

In [57]:
a = [1,2,3,4,5,6,7,8,9]

print(a[1:])

[2, 3, 4, 5, 6, 7, 8, 9]
